# Inserting ISS Data To MySQL with SQLAlchemy

## Collecting data from the API

In [15]:
import requests
url =  "http://api.open-notify.org/iss-now.json"
response = requests.get(url)
json_response = response.json()
json_response

{'message': 'success',
 'timestamp': 1642678223,
 'iss_position': {'longitude': '-140.2382', 'latitude': '19.0535'}}

Extract only the pieces of info we need and store them in a dictionary

In [16]:
data_to_insert = {"latitude":json_response["iss_position"]["latitude"],
                  "longitude":json_response["iss_position"]["longitude"],
                  "iss_timestamp":json_response["timestamp"]}
data_to_insert

{'latitude': '19.0535', 'longitude': '-140.2382', 'iss_timestamp': 1642678223}

Timestamp from UNIX to proper datetime format

In [17]:
from datetime import datetime
data_to_insert["iss_timestamp"] = datetime.utcfromtimestamp(data_to_insert["iss_timestamp"]).strftime('%Y-%m-%d %H:%M:%S')
data_to_insert

{'latitude': '19.0535',
 'longitude': '-140.2382',
 'iss_timestamp': '2022-01-20 11:30:23'}

Create a pandas Dataframe with the same columns as the SQL table we want to have in our database

In [18]:
import pandas as pd

iss_df = pd.DataFrame({
    "latitude":[],
    "longitude":[],
    "iss_timestamp":[]
})
iss_df

,latitude,longitude,iss_timestamp


Append data to DataFrame

In [19]:
iss_df = iss_df.append(data_to_insert, ignore_index=True)
iss_df

,latitude,longitude,iss_timestamp
0,19.0535,-140.2382,2022-01-20 11:30:23


# SQLAlchemy

In [20]:
!pip install PyMySql

In [23]:
import sqlalchemy # install if needed
from api_keys import local_mysql_password

Specify MySQL connection. You need to previously create the schema (also called database in MySQL) `iss_workshop` on your local instance of MySQL. 

You can do that with MySQLWorkbench by connecting to your local instance and typing `CREATE DATABASE iss_workshop;` in a new query tab.

In [24]:
database="iss_workshop"
host="127.0.0.1"
user="donal"
password=local_mysql_password
port=3306
con = f'mysql+pymysql://{user}:{password}@{host}:{port}/{database}'

Use pandas method `to_sql` with the argument `if_exists=append` to create the table (only the first time we run it) and insert the new rows into it.

In [25]:
iss_df.to_sql('iss_position', 
              if_exists='append', 
              con=con, 
              index=False)

NoSuchModuleError: Can't load plugin: sqlalchemy.dialects:mysql.pymysql

Check on MySQLWorkbench that a new table `iss_position` exists within the `iss_workshop` database, and that a new row has been inserted on it. If you run the whole notebook again, another row should appear there.